# Terraria 1.4：为何说专家模式比大师模式困难？单人模式 Boss 血量、攻击数值的初步计算方式

对于该文档的直观认识，可以参考下述视频：[\[泰拉瑞亚1.4\] 大师模式并非最困难模式？！详解旅程模式、For the Worthy、难度滑条对难度的影响](https://www.bilibili.com/video/BV1pA411i75H/)

## 大致结论

首先，我们讨论的是 1.4.0.5 版本的游戏。很可能在以后更新的游戏版本中，下述的结论会产生变化。

我们先开门见山了解一下我所理解的最困难模式为何吧：

**旅程模式 2.95 倍敌怪困难度的 For the Worthy 地图**

我们知道，在 Terraria 1.4 中，旅程模式下的 2.95 倍敌怪困难度并没有达到 3 倍，因此没有踏入大师模式，而仍然应称为专家模式。在这个模式下，玩家会遇到的困难有

- 专家难度无法在血肉墙前获得 6 格饰品栏，以及血肉墙后获得 7 格饰品栏；

- 专家难度无法获得大师模式特有物品，譬如“The Black Spot”；该物品可以较轻松地击败史莱姆女皇与光之女王等敌怪；

- 专家难度 2.95 倍的 Boss 的 HP 并没有因为敌怪困难度而下降，反而是相对于大师难度 3.00 倍上升了 15.7%；

- For the Worthy 本身提升了敌怪的综合性能，包括所有敌怪的 HP、ATK、防御力，以及 Boss 敌怪大小和 AI 本身；

- 旅程模式下的 For the Worthy 比非旅程模式的对应情况要困难至少 2/3；这是指 HP 值与 ATK 值而言，从而容错率非常低。

关于专家模式与大师模式，

- 专家模式与大师模式的 AI 几乎没有区别；

- 一些射弹的伤害上，2 倍难度的专家模式确实比 3 倍难度的大师模式低；但旅程模式的射弹伤害是随着难度滑条的倍数呈线性关系，即 2.95 倍难度的射弹伤害与 3 倍大师难度并没有太大区别。

我们先暂且不谈非 Boss 的敌怪已经相当可怕了——开荒之初 100 点 HP 时，蓝史莱姆两次就可以将玩家踢出世界，而红色史莱姆一次即可。

而对于 Boss 而言，这可能使得从世界吞噬者或克苏鲁之脑后，近乎所有 Boss 敌怪的连续两三次攻击可以让玩家直接出局。同时，Boss 的 HP 值激增，少量的 Boss 的防御值也有所增加，使得攻略 Boss 的容错率变得相当低。或许，一般来说，玩家必须要具有无伤攻略的素质，才能通关这种难度的世界。

当然，这种世界也对应地有一些好处：

- 旅程模式具有研究、复制功能，意味着在得到少量的花草、鱼、矿石等基础材料后，就可以无限制作物品、补药、弹药 (与金钱)；

- For the Worthy 有更多更大的发光蘑菇群落，因此宝箱与生命水晶可能会相对较多且好寻找；

- 2.95 倍难度的 Boss 攻击力、非 Boss 的 HP 与攻击力都比 3.00 倍小出那 0.05 倍。

- 旅程模式的城镇 NPC 能力更强，且敌怪的基础被击退能力与普通模式相同。(专家与大师模式下，敌怪的基础被击退能力都得到了小幅加强)

## 具体过程

In [1]:
import pandas as pd
import numpy as np

### 基础数值

出于简化问题，我们只讨论一部分不会有第二阶段变化的 Boss 型敌怪。我们用 NPC ID 号来称呼它们。

In [2]:
Map_ID_Name = {
    13: "世界吞噬者 (头部)",
    14: "世界吞噬者 (身体)",
    15: "世界吞噬者 (尾部)",
    266: "克苏鲁之脑",
    267: "飞眼怪",
    50: "史莱姆王",
    113: "血肉墙",
    114: "血肉墙 (眼部)",
}

它们的基础 HP 值、攻击力列举如下；基础值的意义是最为普通的普通模式下的数值。从下表能看出，史莱姆王的基础 HP 为 2000、攻击力为 40。从这些数值上看，史莱姆王处于世界吞噬者、克苏鲁之脑之后，蜂王之前确实是合理的。当然，由于史莱姆王的移动方式单一，使得即使 1.4 版本中史诗级地加强了史莱姆王的 AI，它也仍然是最容易攻略的 Boss。

下述数值出自 `Terraria.NPC` 的成员函数 `void SetDefaults` 中。

In [3]:
Base_HP_ATK = {
    13:  ( 150, 22),
    14:  ( 150, 13),
    15:  ( 150, 11),
    266: (1000, 30),
    267: ( 100, 20),
    50:  (2000, 40),
    113: (8000, 50),
    114: (8000, 50),
}

### 专家与大师模式的影响

我们首先定义下述类，该类能通过输入的难度倍数给出当前是否处于专家或大师难度。2.95 倍是专家难度，3 倍是大师难度。

In [4]:
class GameMode:
    
    def __init__(self, power):
        self.power = power
        
    @property
    def isExpert(self):
        return self.power >= 2
    
    @property
    def isMaster(self):
        return self.power >= 3

在 `Terraria.NPC` 的成员函数 `void ScaleStats_ApplyMultiplayerStats` 中，在专家及大师模式下对 Boss 的 HP 与 ATK 值进行了缩放。

In [5]:
GameMode_HP_ATK_Scale = {
    13:  (0.7 , 1.1),
    14:  (0.7 , 0.8),
    15:  (0.7 , 0.8),
    266: (0.85, 0.9),
    267: (0.85, 0.9),
    50:  (0.7 , 0.8),
    113: (0.7 , 1.5),
    114: (0.7 , 1.5),
}

看起来大多数情况下都是削弱而非加强；但在经过上述缩放过程后，还要乘上困难倍数才得到最终的 HP 与 ATK。大师模式的困难倍数是 3 倍。

但如果难度达到大师级别，Boss 的 HP 会回削弱到 0.85 倍。这也就意味着，2.95 倍的专家难度的 Boss HP 会比 3 倍的大师难度更大，且幅度是

2.95 / 3 / 0.85 - 1 = 15.69%

### For the Worthy 的影响

首先，在 `Terraria.NPC` 的成员函数 `void getGoodAdjustments` 中，For the Worthy 会提供一个基础 HP 与 ATK 的基础值的提升：

In [6]:
GoodWorld_HP_ATK_Scale = {
    13:  (1. , 1. ),
    14:  (1. , 1. ),
    15:  (1. , 1. ),
    266: (1. , 1.2),
    267: (1. , 1.2),
    50:  (1. , 1. ),
    113: (1.5, 1. ),
    114: (1.5, 1. ),
}

可以看出，For the Worthy 并没有对许多的 Boss 的基础数值作改动；在这里被加强的 Boss 有克苏鲁之脑 (攻击力加强 20%)，以及血肉墙 (HP 加强 50%)。

随后，在非旅程的专家模式会增加 1/2 倍，大师模式会增加 1/3 倍的 HP 与 ATK 值；旅程模式下都直接增加 1 倍的 HP 与 ATK 值。随后，ATK 值还会额外增加 1/3 倍。这是在 `Terraria.NPC` 的成员函数 `void ScaleStats_ApplyGameMode` 中实现的。

### HP 值总结

我们可以以下述程序来简单地计算不同的模式下，Boss 的 HP 值情况：

In [7]:
def calculate_HP(idx, power, isJourneyMode, isGoodWorld):
    HP = Base_HP_ATK[idx][0]
    if GameMode(idx).isExpert:
        HP *= GameMode_HP_ATK_Scale[idx][0]
    if GameMode(power).isMaster:
        HP *= 0.85
    if isGoodWorld:
        HP *= GoodWorld_HP_ATK_Scale[idx][0]
        if not isJourneyMode:
            HP *= (power + 1)
        else:
            HP *= 2 * power
    else:
        HP *= power
    return int(HP)

譬如，对于克苏鲁之脑，其在 2.95 倍难度的旅程模式 For the Worthy 下，其 HP 值为

In [8]:
calculate_HP(266, 2.95, True, True)

5015

这个血量值已经接近通常认为最为困难的大师模式 For the Worthy 的两倍了。

我们用下述的 pandas 表格呈现结果：

In [9]:
header = pd.MultiIndex.from_product([
    ["普通地图", "For the Worthy"],
    ["非旅程模式", "旅程模式"],
    ["专家", "2.95", "大师"]],
    names=["HP 值", "", "困难度"])

In [10]:
data = np.array([[[[
    calculate_HP(idx, power, isJourneyMode, isGoodWorld)
    for power in (2, 2.95, 3)]
    for isJourneyMode in (False, True)]
    for isGoodWorld in (False, True)]
    for idx in Map_ID_Name]
).reshape(-1, 3 * 2 * 2)

In [11]:
df = pd.DataFrame(data, index=[val for val in Map_ID_Name.values()], columns=header)
df

HP 值         普通地图                                    For the Worthy         \
            非旅程模式                 旅程模式                        非旅程模式          
困难度            专家   2.95     大师     专家   2.95     大师             专家   2.95   
世界吞噬者 (头部)    210    309    267    210    309    267            315    414   
世界吞噬者 (身体)    210    309    267    210    309    267            315    414   
世界吞噬者 (尾部)    210    309    267    210    309    267            315    414   
克苏鲁之脑        1700   2507   2167   1700   2507   2167           2550   3357   
飞眼怪           170    250    216    170    250    216            255    335   
史莱姆王         2800   4130   3570   2800   4130   3570           4200   5530   
血肉墙         11200  16520  14280  11200  16520  14280          25200  33180   
血肉墙 (眼部)    11200  16520  14280  11200  16520  14280          25200  33180   

HP 值                                    
                    旅程模式                
困难度            大师     专家   2.95     大师  
世界吞噬者 (头部)    357    420    619    535  
世界吞噬者 (身体)    357    420    619    535  
世界吞噬者 (尾部)    357    420    619    535  
克苏鲁之脑        2890   3400   5015   4335  
飞眼怪           289    340    501    433  
史莱姆王         4760   5600   8260   7140  
血肉墙         28560  33600  49560  42840  
血肉墙 (眼部)    28560  33600  49560  42840

### ATK 值总结

我们可以以下述程序来简单地计算不同的模式下，Boss 的 ATK 值情况：

In [12]:
def calculate_ATK(idx, power, isJourneyMode, isGoodWorld):
    ATK = Base_HP_ATK[idx][1]
    if GameMode(idx).isExpert:
        ATK *= GameMode_HP_ATK_Scale[idx][1]
    if isGoodWorld:
        ATK *= GoodWorld_HP_ATK_Scale[idx][1]
        if not isJourneyMode:
            ATK *= (power + 1)
        else:
            ATK *= 2 * power
        ATK *= 4/3
    else:
        ATK *= power
    return int(ATK)

譬如，对于血肉墙，其在 2.95 倍难度的旅程模式 For the Worthy 下，其 ATK 值为

In [13]:
calculate_ATK(113, 2.95, True, True)

590

这也就意味着，不论玩家防御力多高，如果只有血肉墙前的装备，在其贴身一击下，是不可能活下来的。

我们用下述的 pandas 表格呈现结果：

In [14]:
header = pd.MultiIndex.from_product([
    ["普通地图", "For the Worthy"],
    ["非旅程模式", "旅程模式"],
    ["专家", "2.95", "大师"]],
    names=["ATK 值", "", "困难度"])

In [15]:
data = np.array([[[[
    calculate_ATK(idx, power, isJourneyMode, isGoodWorld)
    for power in (2, 2.95, 3)]
    for isJourneyMode in (False, True)]
    for isGoodWorld in (False, True)]
    for idx in Map_ID_Name]
).reshape(-1, 3 * 2 * 2)

In [16]:
df = pd.DataFrame(data, index=[val for val in Map_ID_Name.values()], columns=header)
df

ATK 值       普通地图                          For the Worthy                      \
           非旅程模式           旅程模式                    非旅程模式           旅程模式        
困难度           专家 2.95   大师   专家 2.95   大师             专家 2.95   大师   专家 2.95   
世界吞噬者 (头部)    48   71   72   48   71   72             96  127  129  129  190   
世界吞噬者 (身体)    20   30   31   20   30   31             41   54   55   55   81   
世界吞噬者 (尾部)    17   25   26   17   25   26             35   46   46   46   69   
克苏鲁之脑         54   79   81   54   79   81            129  170  172  172  254   
飞眼怪           36   53   54   36   53   54             86  113  115  115  169   
史莱姆王          64   94   96   64   94   96            128  168  170  170  251   
血肉墙          150  221  225  150  221  225            300  395  400  400  590   
血肉墙 (眼部)     150  221  225  150  221  225            300  395  400  400  590   

ATK 值            
                 
困难度          大师  
世界吞噬者 (头部)  193  
世界吞噬者 (身体)   83  
世界吞噬者 (尾部)   70  
克苏鲁之脑       259  
飞眼怪         172  
史莱姆王        256  
血肉墙         600  
血肉墙 (眼部)    600

## 补充信息

上述程序信息来自 [ILSpy](https://github.com/icsharpcode/ILSpy) 对 Terraria 1.4.0.5 的反向工程结果。